This script generates N ("it" variable) velocity files and for each runs surfacevel2strain (Tape et al., 2009).

Input: 
- Before running this script, parameters adapted to the studied region need to be set in "surfacevel2strain.m" and "get_gps_dataset.m" (see description of the parameters in 03_runmatlab_surfacevel2strain_multiplevelfiles script, "time_dependent_strainrate" repository)
- Path to a folder with files generated by surfacevel2strain
Output:
- N folders (it_N/) containing surfacevel2strain output files

In [4]:
import re,os,subprocess,glob
import shutil
import numpy as np
import math
import matlab.engine

In [5]:
# the variables outdir and veldir correspond to the path to surfacevel2strain output and input velocity file 
outdir="/home/gmeneses/local/compearth-master/surfacevel2strain/matlab_output/current_test_results" 
veldir="/home/gmeneses/local/compearth-master/surfacevel2strain/data/ITALY"
# path to folder that contains surfacevel2strain output files
velfile="cGPS_GrAtSiD_velocities_IGS14_italy_2010.0_2012.11_compearth_format.vel" 
velfilepath="/home/gmeneses/Documents/RUB/Research/05_Italy_NGL_decomposition_Gianina/overlapped_window_3y_Gratsidvel/test_20100_201211/"+velfile

In [6]:
localdir=os.getcwd()
eng = matlab.engine.start_matlab()
#this is the folder path of surfacevel2strain folders containing matlab scripts
s = eng.genpath('/home/gmeneses/local/compearth-master/surfacevel2strain')
eng.addpath(s, nargout=0)

# a new velocity file is created in each iteration. New velocities vary according observations uncertainties.
# working with more than 100 iterations (modify "it" variable) is computationally slow at the moment
it = 5
for i in range(0,it):
    ofile=open("test.dat","w")
    with open(velfilepath,"r") as orig:
        for line in orig:
            l = line.rstrip()
            lon=l.split()[0]
            lat=l.split()[1]
            ve=l.split()[2]
            vn=l.split()[3]
            vu=l.split()[4]
            ee=l.split()[5]
            en=l.split()[6]
            eu=l.split()[7]
            di=l.split()[-3]
            df=l.split()[-2]
            sta=l.split()[-1]
            mue, sigmae = float(ve),float(ee)
            mun, sigman = float(vn),float(en)
            muu, sigmau = float(vu),float(eu)
            se = np.random.normal(mue, sigmae, size=None)
            sn = np.random.normal(mun, sigman, size=None)
            su = np.random.normal(muu, sigmau, size=None)
            print(lon,lat,se,sn,su,ee,en,eu,"0 0 0",di,df,sta,file=ofile)
        ofile.close()    
        shutil.move(localdir+"/test.dat",veldir+"/italy.vel")
    
        eng.surfacevel2strain(nargout=0)

        source = outdir + "/"
        dest = localdir+ "/it_" +str(i) + "/"
        
        os.makedirs(dest,exist_ok=True)
        shutil.move(veldir+"/italy.vel",dest+velfile)
        allfiles = os.listdir(source)
        for f in allfiles:
            shutil.move(source + f, dest + f)